# 데이터 불러오기

In [95]:
# 데이터 처리 및 분석
import numpy as np
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', None)  # 모든 컬럼 다 보이게 설정

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from scipy.stats import shapiro

# 지리 라이브러리
from geopy.distance import geodesic

# 전처리용
from itertools import combinations

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 윈도우용 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')  # 말굿 고딕 (Windows 기본 한글 폰트)
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 머신러닝 - 전처리, 모델, 평가
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# 머신러닝 알고리즘 (필요에 따라 선택적으로 추가)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier, XGBRegressor

# 텍스트 처리
from rapidfuzz import process, fuzz

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# # # 딥러닝 (선택사항)
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

# 모델 해석
import shap



In [96]:
subway_test_before  = pd.read_excel('./data/test_subway.xlsx')
air_test_before     = pd.read_csv('./data/test_air.csv', encoding='cp949')
weather_test_before = pd.read_excel('./data/test_weather.xlsx')
subway_train_before = pd.read_csv('./data/train_subway.csv', encoding='cp949')
air_train_before    = pd.read_csv('./data/train_air.csv', encoding='cp949')
weather_train_before = pd.read_csv('./data/train_weather.csv', encoding='cp949')

In [97]:
air_train_before['공기질'].unique()

array([ 0.,  1., nan])

In [98]:
# 전처리 함수 정의
def preprocess_air(df):
    df['일시'] = pd.to_datetime(df['일시'], format='%Y-%m-%d %H')
    df['역명_호선'] = df['역명'] + "_" + df['호선'].astype(str)
    return df[['일시', '역명_호선', '공기질']]

def preprocess_air(df):
    # 일시 처리
    df['일시'] = pd.to_datetime(df['일시'], format='%Y-%m-%d %H')
    df['역명_호선'] = df['역명'] + "_" + df['호선'].astype(str)
    
    # 전체 시간 범위 생성
    full_time_range = pd.date_range(start=df['일시'].min(), end=df['일시'].max(), freq='H')

    # 역명_호선별로 모든 시간대 확보
    역호선_리스트 = df['역명_호선'].unique()
    full_index = pd.MultiIndex.from_product([full_time_range, 역호선_리스트], names=['일시', '역명_호선'])

    # 원본을 재구성하여 누락된 시간대 삽입
    df_result = df.set_index(['일시', '역명_호선'])[['공기질']].reindex(full_index).reset_index()

    return df_result



def preprocess_subway(df):
    df['수송일자'] = pd.to_datetime(df['수송일자'])
    df['역명_호선'] = df['역명'] + "_" + df['호선'].str.extract(r'(\d+)호선')[0]
    time_cols = [col for col in df.columns if '시' in col or '시간대' in col]
    df_melted = df.melt(id_vars=['수송일자', '역명_호선'], value_vars=time_cols, var_name='시간', value_name='승하차수')
    df_melted['시간'] = df_melted['시간'].str.extract(r'(\d+)')[0].astype(float).fillna(0).astype(int)
    df_melted['일시'] = df_melted['수송일자'] + pd.to_timedelta(df_melted['시간'], unit='h')
    df_grouped = df_melted.groupby(['일시', '역명_호선'])['승하차수'].sum().reset_index()
    return df_grouped

def preprocess_subway(df):
    df['수송일자'] = pd.to_datetime(df['수송일자'])
    df['역명_호선'] = df['역명'] + "_" + df['호선'].str.extract(r'(\d+)호선')[0]

    time_cols = [col for col in df.columns if '시' in col or '시간대' in col]
    df_melted = df.melt(
        id_vars=['수송일자', '역명_호선'],
        value_vars=time_cols,
        var_name='시간',
        value_name='승하차수'
    )

    # 시간 문자열 처리
    df_melted['시간_str'] = df_melted['시간']  # 원본 보존
    df_melted['시간'] = df_melted['시간'].str.extract(r'(\d+)')[0].astype(float).fillna(0).astype(int)

    # 06시이전은 5시로, 24시이후는 24시로 고정
    df_melted.loc[df_melted['시간_str'].str.contains('06시이전'), '시간'] = 5
    df_melted.loc[df_melted['시간_str'].str.contains('24시이후'), '시간'] = 0

    # 일시 계산: 시간 기준으로 일단 더하고
    df_melted['일시'] = df_melted['수송일자'] + pd.to_timedelta(df_melted['시간'], unit='h')

    # 24시이후는 +1일 조정 (이미 시간은 0시로 설정했음)
    is_after_midnight = df_melted['시간_str'].str.contains('24시이후')
    df_melted.loc[is_after_midnight, '일시'] += pd.Timedelta(days=1)

    # 최종 집계
    df_grouped = df_melted.groupby(['일시', '역명_호선'])['승하차수'].sum().reset_index()
    return df_grouped

def preprocess_subway(df):
    df['수송일자'] = pd.to_datetime(df['수송일자'])
    df['역명_호선'] = df['역명'] + "_" + df['호선'].str.extract(r'(\d+)호선')[0]

    # melt 대상 컬럼 선택
    time_cols = [col for col in df.columns if '시' in col or '시간대' in col]

    if '승하차구분' in df.columns:
        df_melted = df.melt(
            id_vars=['수송일자', '역명_호선', '승하차구분'],
            value_vars=time_cols,
            var_name='시간',
            value_name='승하차수'
        )
    else:
        df['승하차구분'] = '총계'
        df_melted = df.melt(
            id_vars=['수송일자', '역명_호선', '승하차구분'],
            value_vars=time_cols,
            var_name='시간',
            value_name='승하차수'
        )

    # 시간 처리
    df_melted['시간_str'] = df_melted['시간']
    df_melted['시간'] = df_melted['시간'].str.extract(r'(\d+)')[0].astype(float).fillna(0).astype(int)

    df_melted.loc[df_melted['시간_str'].str.contains('06시이전'), '시간'] = 5
    df_melted.loc[df_melted['시간_str'].str.contains('24시이후'), '시간'] = 0

    df_melted['일시'] = df_melted['수송일자'] + pd.to_timedelta(df_melted['시간'], unit='h')
    df_melted.loc[df_melted['시간_str'].str.contains('24시이후'), '일시'] += pd.Timedelta(days=1)

    # 피벗: 승차 / 하차 구분이 있을 때만
    df_pivot = df_melted.pivot_table(
        index=['일시', '역명_호선'],
        columns='승하차구분',
        values='승하차수',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    df_pivot.columns.name = None

    # 컬럼 명시적으로 정리
    if '승차' in df_pivot.columns and '하차' in df_pivot.columns:
        df_pivot = df_pivot.rename(columns={'승차': '승차수', '하차': '하차수'})
        df_pivot['승하차수'] = df_pivot['승차수'] + df_pivot['하차수']
    else:
        df_pivot['승차수'] = 0
        df_pivot['하차수'] = 0
        df_pivot = df_pivot.rename(columns={'총계': '승하차수'})
    df_pivot['승차수'] = df_pivot['승차수'].fillna(0)
    df_pivot['하차수'] = df_pivot['하차수'].fillna(0)

    return df_pivot




def preprocess_weather(df):
    df['일시'] = pd.to_datetime(df['일시'], format='%Y-%m-%d %H:%M')
    df = df.fillna(0)
    return df

In [99]:
# 전처리 적용
air_train = preprocess_air(air_train_before)
air_test = preprocess_air(air_test_before)
subway_train = preprocess_subway(subway_train_before)
subway_test = preprocess_subway(subway_test_before)
weather_train = preprocess_weather(weather_train_before)
weather_test = preprocess_weather(weather_test_before)

In [100]:
air_train['공기질'].unique(), air_train.shape

(array([ 0.,  1., nan]), (87600, 3))

In [101]:
# 통합 함수
def merge_data(air, subway, weather):
    # 일시 컬럼 타입 통일
    air['일시'] = pd.to_datetime(air['일시'])
    subway['일시'] = pd.to_datetime(subway['일시'])
    weather['일시'] = pd.to_datetime(weather['일시'])

    # 병합 수행
    df = air.merge(subway, on=['일시', '역명_호선'], how='left')
    df = df.merge(weather, on='일시', how='left')

    df['승하차수'] = df['승하차수'].fillna(0)

    df['승차수'] = df['승차수'].fillna(0)

    df['하차수'] = df['하차수'].fillna(0)
    # df = df.dropna()

    return df

train_df = merge_data(air_train, subway_train, weather_train)
test_df = merge_data(air_test, subway_test, weather_test)

In [102]:
holidays_df = pd.read_csv('./data2/holidays.csv')

In [103]:
# 날짜관련 feature
def add_date_features(train_df: pd.DataFrame, holidays_df: pd.DataFrame) -> pd.DataFrame:
    df = train_df.copy()
    
    # holidays 처리
    holidays_df['date'] = pd.to_datetime(holidays_df['date']).dt.date
    holidays_df = holidays_df.drop_duplicates()

    # 날짜 컬럼 생성
    df['date'] = df['일시'].dt.date
    df['weekday'] = df['일시'].dt.weekday  # Monday=0, Sunday=6

    # 휴일 및 주말 여부
    df['holi_yn'] = df['date'].isin(holidays_df['date']).astype(int)
    df['sat_yn'] = (df['weekday'] == 5).astype(int)
    df['sun_yn'] = (df['weekday'] == 6).astype(int)
    df['workday_yn'] = ((df['holi_yn'] == 0) & (df['sun_yn'] == 0) & (df['sat_yn'] == 0)).astype(int)
    df['holiday_yn'] = ((df['holi_yn'] == 1) | (df['sun_yn'] == 1) | (df['sat_yn'] == 1)).astype(int)

    # 날짜 기준 연속 일수 계산 함수
    def compute_streak_by_date(df: pd.DataFrame, binary_col: str) -> pd.Series:
        daily_status = df.groupby(df['일시'].dt.date)[binary_col].max()
        block = (daily_status != daily_status.shift()).cumsum()
        daily_streak = daily_status.groupby(block).cumsum() * daily_status
        date_to_streak = daily_streak.to_dict()
        return df['일시'].dt.date.map(date_to_streak)

    df['workday_continue'] = compute_streak_by_date(df, 'workday_yn')
    df['holiday_continue'] = compute_streak_by_date(df, 'holiday_yn')

    return df


In [104]:
train_df = add_date_features(train_df, holidays_df)
test_df = add_date_features(test_df, holidays_df)

In [105]:
def add_time_cycles(df):
    df = df.copy()

    df['hour'] = df['일시'].dt.hour
    df['weekday'] = df['일시'].dt.weekday
    df['month'] = df['일시'].dt.month
    df['quarter'] = df['일시'].dt.quarter

    # 시간 (24시간)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

    # 요일 (7일)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)

    # 월 (12개월)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # 분기 (4분기)
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)

    return df

In [106]:
train_df = add_time_cycles(train_df)
test_df = add_time_cycles(test_df)

In [107]:
def add_subway_lag_features(df):
    """
    역사별 시간순 정렬 기준으로 승차수, 하차수, 승하차수에 대해
    1, 2, 3시간 전 lag 컬럼과 최근 3시간 평균, 비율, 증감률을 계산합니다.
    """
    df = df.copy()
    
    # 정렬
    df = df.sort_values(['역명_호선', '일시'])
    
    # lag 컬럼 생성
    for i in [1, 2, 3]:
        df[f'승차수_lag{i}'] = df.groupby('역명_호선')['승차수'].shift(i)
        df[f'하차수_lag{i}'] = df.groupby('역명_호선')['하차수'].shift(i)
        df[f'승하차수_lag{i}'] = df.groupby('역명_호선')['승하차수'].shift(i)
    
    # 최근 3시간 평균
    df['승하차수_최근3시간평균'] = df[['승하차수_lag1', '승하차수_lag2', '승하차수_lag3']].mean(axis=1)
    
    # 비율 및 증감률 계산
    df['승하차수_최근3시간비율'] = df['승하차수'] / df['승하차수_최근3시간평균']
    df['승하차수_최근3시간증감률(%)'] = ((df['승하차수'] - df['승하차수_최근3시간평균']) / df['승하차수_최근3시간평균']) * 100

    # 0으로 나눔 방지
    df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

    return df


In [108]:
train_df = add_subway_lag_features(train_df)
test_df = add_subway_lag_features(test_df)

* 기상 lag

In [109]:
# 기상 관련 변수 추출 (온도, 기압, 습도 등으로 추정되는 변수들)
weather_cols = [
    '기온', '강수량', '풍속', '습도', '증기압', '이슬점온도',
    '현지기압', '해면기압', '일조', '일사', '적설',
    '지면온도', '5cm지중온도', '10cm지중온도', '20cm지중온도', '30cm지중온도'
]
weather_cols = [
    "기온(°C)",    "강수량(mm)",    "풍속(m/s)",    "습도(%)",    "증기압(hPa)",    "이슬점온도(°C)",
    "현지기압(hPa)",    "해면기압(hPa)",    "일조(hr)",    "일사(MJ/m2)",    "적설(cm)",
    "지면온도(°C)",    "5cm 지중온도(°C)",    "10cm 지중온도(°C)",    "20cm 지중온도(°C)",    "30cm 지중온도(°C)"
]

group_cols = ['역명_호선']

In [110]:
# h-3 lag 피처 생성 함수 정의
def add_weather_lag_features(df, weather_cols, group_col, lag_hours=3):
    df = df.sort_values(by=['역명_호선', '일시'])
    for col in weather_cols:
        lag_col = f'{col}_lag{lag_hours}'
        df[lag_col] = df.groupby(group_col)[col].shift(lag_hours)
    return df

# 3시간 평균, 비율, 변화량을 생성하는 함수 정의
def add_weather_rolling_features(df, weather_cols, group_col, window=3):
    df = df.sort_values(by=[group_col, '일시'])
    for col in weather_cols:
        # 최근 3시간 평균
        df[f'{col}_mean{window}h'] = df.groupby(group_col)[col].rolling(window).mean().reset_index(level=0, drop=True)
        # 최근 3시간 비율: 현재값 / 평균
        df[f'{col}_ratio{window}h'] = df[col] / df[f'{col}_mean{window}h']
        # 변화량: 현재값 - 3시간 전
        df[f'{col}_diff{window}h'] = df[col] - df.groupby(group_col)[col].shift(window)
    return df

In [111]:
train_df = add_weather_lag_features(train_df.copy(), weather_cols, group_col='역명_호선', lag_hours=1)
test_df = add_weather_lag_features(test_df.copy(), weather_cols, group_col='역명_호선', lag_hours=1)

train_df = add_weather_lag_features(train_df.copy(), weather_cols, group_col='역명_호선', lag_hours=2)
test_df = add_weather_lag_features(test_df.copy(), weather_cols, group_col='역명_호선', lag_hours=2)

train_df = add_weather_lag_features(train_df.copy(), weather_cols, group_col='역명_호선', lag_hours=3)
test_df = add_weather_lag_features(test_df.copy(), weather_cols, group_col='역명_호선', lag_hours=3)

# 기존에 lag 3까지 반영된 데이터에 rolling 기반 피처 추가
train_df = add_weather_rolling_features(train_df, weather_cols, group_col='역명_호선', window=3)
test_df = add_weather_rolling_features(test_df  , weather_cols, group_col='역명_호선', window=3)


- 열차 스케츌

In [112]:
schedule_df = pd.read_csv('./data2/subway_schedule.csv')

In [113]:
def merge_subway_schedule(train_df: pd.DataFrame, schedule_df: pd.DataFrame) -> pd.DataFrame:
    # 1. 주중주말 컬럼 생성 (공휴일 > 일요일 > 토요일 > 평일)
    def classify_day_from_flags(row):
        if row['holi_yn'] == 1:
            return 'END'
        elif row['sun_yn'] == 1:
            return 'END'
        elif row['sat_yn'] == 1:
            return 'SAT'
        else:
            return 'DAY'

    train_df = train_df.copy()
    train_df['주중주말'] = train_df.apply(classify_day_from_flags, axis=1)

    # 2. 일시로부터 '시간' 추출
    train_df['시간'] = train_df['일시'].dt.hour

    # 3. 역명_호선 분리 → 역사명, 호선
    train_df[['역사명', '호선']] = train_df['역명_호선'].str.extract(r'(.+)_([0-9]+)')

    # 4. 데이터 타입 정리 (schedule 쪽과 호선/시간이 일치해야 함)
    train_df['호선'] = train_df['호선'].astype(str)
    train_df['시간'] = train_df['시간'].astype(int)
    schedule_df['호선'] = schedule_df['호선'].astype(str)
    schedule_df['시간'] = schedule_df['시간'].astype(int)

    # 5. 병합 수행
    merged = pd.merge(
        train_df,
        schedule_df,
        how='left',
        on=['호선', '역사명', '주중주말', '시간']
    )
    merged['운행횟수'] = merged['운행횟수'].fillna(0)
    
    return merged

In [114]:
merge_subway_schedule(train_df, schedule_df)

,일시,역명_호선,공기질,승차수,하차수,승하차수,기온(°C),강수량(mm),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),date,weekday,holi_yn,sat_yn,sun_yn,workday_yn,holiday_yn,workday_continue,holiday_continue,hour,month,quarter,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos,quarter_sin,quarter_cos,승차수_lag1,하차수_lag1,승하차수_lag1,승차수_lag2,하차수_lag2,승하차수_lag2,승차수_lag3,하차수_lag3,승하차수_lag3,승하차수_최근3시간평균,승하차수_최근3시간비율,승하차수_최근3시간증감률(%),기온(°C)_lag1,강수량(mm)_lag1,풍속(m/s)_lag1,습도(%)_lag1,증기압(hPa)_lag1,이슬점온도(°C)_lag1,현지기압(hPa)_lag1,해면기압(hPa)_lag1,일조(hr)_lag1,일사(MJ/m2)_lag1,적설(cm)_lag1,지면온도(°C)_lag1,5cm 지중온도(°C)_lag1,10cm 지중온도(°C)_lag1,20cm 지중온도(°C)_lag1,30cm 지중온도(°C)_lag1,기온(°C)_lag2,강수량(mm)_lag2,풍속(m/s)_lag2,습도(%)_lag2,증기압(hPa)_lag2,이슬점온도(°C)_lag2,현지기압(hPa)_lag2,해면기압(hPa)_lag2,일조(hr)_lag2,일사(MJ/m2)_lag2,적설(cm)_lag2,지면온도(°C)_lag2,5cm 지중온도(°C)_lag2,10cm 지중온도(°C)_lag2,20cm 지중온도(°C)_lag2,30cm 지중온도(°C)_lag2,기온(°C)_lag3,강수량(mm)_lag3,풍속(m/s)_lag3,습도(%)_lag3,증기압(hPa)_lag3,이슬점온도(°C)_lag3,현지기압(hPa)_lag3,해면기압(hPa)_lag3,일조(hr)_lag3,일사(MJ/m2)_lag3,적설(cm)_lag3,지면온도(°C)_lag3,5cm 지중온도(°C)_lag3,10cm 지중온도(°C)_lag3,20cm 지중온도(°C)_lag3,30cm 지중온도(°C)_lag3,기온(°C)_mean3h,기온(°C)_ratio3h,기온(°C)_diff3h,강수량(mm)_mean3h,강수량(mm)_ratio3h,강수량(mm)_diff3h,풍속(m/s)_mean3h,풍속(m/s)_ratio3h,풍속(m/s)_diff3h,습도(%)_mean3h,습도(%)_ratio3h,습도(%)_diff3h,증기압(hPa)_mean3h,증기압(hPa)_ratio3h,증기압(hPa)_diff3h,이슬점온도(°C)_mean3h,이슬점온도(°C)_ratio3h,이슬점온도(°C)_diff3h,현지기압(hPa)_mean3h,현지기압(hPa)_ratio3h,현지기압(hPa)_diff3h,해면기압(hPa)_mean3h,해면기압(hPa)_ratio3h,해면기압(hPa)_diff3h,일조(hr)_mean3h,일조(hr)_ratio3h,일조(hr)_diff3h,일사(MJ/m2)_mean3h,일사(MJ/m2)_ratio3h,일사(MJ/m2)_diff3h,적설(cm)_mean3h,적설(cm)_ratio3h,적설(cm)_diff3h,지면온도(°C)_mean3h,지면온도(°C)_ratio3h,지면온도(°C)_diff3h,5cm 지중온도(°C)_mean3h,5cm 지중온도(°C)_ratio3h,5cm 지중온도(°C)_diff3h,10cm 지중온도(°C)_mean3h,10cm 지중온도(°C)_ratio3h,10cm 지중온도(°C)_diff3h,20cm 지중온도(°C)_mean3h,20cm 지중온도(°C)_ratio3h,20cm 지중온도(°C)_diff3h,30cm 지중온도(°C)_mean3h,30cm 지중온도(°C)_ratio3h,30cm 지중온도(°C)_diff3h,주중주말,시간,역사명,호선,운행횟수
0,2022-01-01 00:00:00,강남_2,0.0,0.0,0.0,0.0,-8.5,0.0,1.9,41.0,1.3,-19.4,1021.9,1033.2,0.0,0.0,0.0,-7.0,-1.0,-1.0,-0.2,1.0,2022-01-01,5,1,1,0,0,1,0,1,0,1,1,0.000000,1.000000,-0.974928,-0.222521,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END,0,강남,2,0.0
1,2022-01-01 01:00:00,강남_2,1.0,0.0,0.0,0.0,-9.2,0.0,1.8,42.0,1.3,-19.7,1022.2,1033.6,0.0,0.0,0.0,-7.2,-1.1,-1.1,-0.2,1.0,2022-01-01,5,1,1,0,0,1,0,1,1,1,1,0.258819,0.965926,-0.974928,-0.222521,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,-8.5,0.0,1.9,41.0,1.3,-19.4,1021.9,1033.2,0.0,0.00,0.0,-7.0,-1.0,-1.0,-0.2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END,1,강남,2,0.0
2,2022-01-01 02:00:00,강남_2,0.0,0.0,0.0,0.0,-9.5,0.0,1.2,43.0,1.3,-19.7,1022.3,1033.7,0.0,0.0,0.0,-7.5,-1.3,-1.2,-0.2,1.0,2022-01-01,5,1,1,0,0,1,0,1,2,1,1,0.500000,0.866025,-0.974928,-0.222521,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,-9.2,0.0,1.8,42.0,1.3,-19.7,1022.2,1033.6,0.0,0.00,0.0,-7.2,-1.1,-1.1,-0.2,1.0,-8.5,0.0,1.9,41.0,1.3,-19.4,1021.9,1033.2,0.0,0.00,0.0,-7.0,-1.0,-1.0,-0.2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.066667,1.047794,NaN,0.0,NaN,NaN,1.

In [115]:
# 함수 실행
train_df = merge_subway_schedule(train_df, schedule_df)
# 함수 실행
test_df = merge_subway_schedule(test_df, schedule_df)



In [116]:
def add_congestion_variable(df):
    df['혼잡도'] = df.apply(
        lambda row: row['승하차수'] / row['운행횟수'] if pd.notnull(row['운행횟수']) and row['운행횟수'] > 0 else 0,
        axis=1
    )
    return df


In [117]:
# 함수 실행
train_df = add_congestion_variable(train_df)
# 함수 실행
test_df = add_congestion_variable(test_df)

train_df

,일시,역명_호선,공기질,승차수,하차수,승하차수,기온(°C),강수량(mm),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),date,weekday,holi_yn,sat_yn,sun_yn,workday_yn,holiday_yn,workday_continue,holiday_continue,hour,month,quarter,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos,quarter_sin,quarter_cos,승차수_lag1,하차수_lag1,승하차수_lag1,승차수_lag2,하차수_lag2,승하차수_lag2,승차수_lag3,하차수_lag3,승하차수_lag3,승하차수_최근3시간평균,승하차수_최근3시간비율,승하차수_최근3시간증감률(%),기온(°C)_lag1,강수량(mm)_lag1,풍속(m/s)_lag1,습도(%)_lag1,증기압(hPa)_lag1,이슬점온도(°C)_lag1,현지기압(hPa)_lag1,해면기압(hPa)_lag1,일조(hr)_lag1,일사(MJ/m2)_lag1,적설(cm)_lag1,지면온도(°C)_lag1,5cm 지중온도(°C)_lag1,10cm 지중온도(°C)_lag1,20cm 지중온도(°C)_lag1,30cm 지중온도(°C)_lag1,기온(°C)_lag2,강수량(mm)_lag2,풍속(m/s)_lag2,습도(%)_lag2,증기압(hPa)_lag2,이슬점온도(°C)_lag2,현지기압(hPa)_lag2,해면기압(hPa)_lag2,일조(hr)_lag2,일사(MJ/m2)_lag2,적설(cm)_lag2,지면온도(°C)_lag2,5cm 지중온도(°C)_lag2,10cm 지중온도(°C)_lag2,20cm 지중온도(°C)_lag2,30cm 지중온도(°C)_lag2,기온(°C)_lag3,강수량(mm)_lag3,풍속(m/s)_lag3,습도(%)_lag3,증기압(hPa)_lag3,이슬점온도(°C)_lag3,현지기압(hPa)_lag3,해면기압(hPa)_lag3,일조(hr)_lag3,일사(MJ/m2)_lag3,적설(cm)_lag3,지면온도(°C)_lag3,5cm 지중온도(°C)_lag3,10cm 지중온도(°C)_lag3,20cm 지중온도(°C)_lag3,30cm 지중온도(°C)_lag3,기온(°C)_mean3h,기온(°C)_ratio3h,기온(°C)_diff3h,강수량(mm)_mean3h,강수량(mm)_ratio3h,강수량(mm)_diff3h,풍속(m/s)_mean3h,풍속(m/s)_ratio3h,풍속(m/s)_diff3h,습도(%)_mean3h,습도(%)_ratio3h,습도(%)_diff3h,증기압(hPa)_mean3h,증기압(hPa)_ratio3h,증기압(hPa)_diff3h,이슬점온도(°C)_mean3h,이슬점온도(°C)_ratio3h,이슬점온도(°C)_diff3h,현지기압(hPa)_mean3h,현지기압(hPa)_ratio3h,현지기압(hPa)_diff3h,해면기압(hPa)_mean3h,해면기압(hPa)_ratio3h,해면기압(hPa)_diff3h,일조(hr)_mean3h,일조(hr)_ratio3h,일조(hr)_diff3h,일사(MJ/m2)_mean3h,일사(MJ/m2)_ratio3h,일사(MJ/m2)_diff3h,적설(cm)_mean3h,적설(cm)_ratio3h,적설(cm)_diff3h,지면온도(°C)_mean3h,지면온도(°C)_ratio3h,지면온도(°C)_diff3h,5cm 지중온도(°C)_mean3h,5cm 지중온도(°C)_ratio3h,5cm 지중온도(°C)_diff3h,10cm 지중온도(°C)_mean3h,10cm 지중온도(°C)_ratio3h,10cm 지중온도(°C)_diff3h,20cm 지중온도(°C)_mean3h,20cm 지중온도(°C)_ratio3h,20cm 지중온도(°C)_diff3h,30cm 지중온도(°C)_mean3h,30cm 지중온도(°C)_ratio3h,30cm 지중온도(°C)_diff3h,주중주말,시간,역사명,호선,운행횟수,혼잡도
0,2022-01-01 00:00:00,강남_2,0.0,0.0,0.0,0.0,-8.5,0.0,1.9,41.0,1.3,-19.4,1021.9,1033.2,0.0,0.0,0.0,-7.0,-1.0,-1.0,-0.2,1.0,2022-01-01,5,1,1,0,0,1,0,1,0,1,1,0.000000,1.000000,-0.974928,-0.222521,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END,0,강남,2,0.0,0.000000
1,2022-01-01 01:00:00,강남_2,1.0,0.0,0.0,0.0,-9.2,0.0,1.8,42.0,1.3,-19.7,1022.2,1033.6,0.0,0.0,0.0,-7.2,-1.1,-1.1,-0.2,1.0,2022-01-01,5,1,1,0,0,1,0,1,1,1,1,0.258819,0.965926,-0.974928,-0.222521,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,-8.5,0.0,1.9,41.0,1.3,-19.4,1021.9,1033.2,0.0,0.00,0.0,-7.0,-1.0,-1.0,-0.2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,END,1,강남,2,0.0,0.000000
2,2022-01-01 02:00:00,강남_2,0.0,0.0,0.0,0.0,-9.5,0.0,1.2,43.0,1.3,-19.7,1022.3,1033.7,0.0,0.0,0.0,-7.5,-1.3,-1.2,-0.2,1.0,2022-01-01,5,1,1,0,0,1,0,1,2,1,1,0.500000,0.866025,-0.974928,-0.222521,5.000000e-01,0.866025,1.000000e+00,6.123234e-17,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,-9.2,0.0,1.8,42.0,1.3,-19.7,1022.2,1033.6,0.0,0.00,0.0,-7.2,-1.1,-1.1,-0.2,1.0,-8.5,0.0,1.9,41.0,1.3,-19.4,1021.9,1033.2,0.0,0.00,0.0,-7.0,-1.0,-1.0,-0.2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.066667,1.047

In [118]:
def add_difference_column(df1: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    result = df1.copy()
    new_col = f"{col1}_minus_{col2}"
    result[new_col] = result[col1] - result[col2]
    return result


In [119]:
train_df = add_difference_column(train_df, '현지기압(hPa)', '해면기압(hPa)')
test_df = add_difference_column(test_df,   '현지기압(hPa)', '해면기압(hPa)')
train_df = add_difference_column(train_df, '5cm 지중온도(°C)', '30cm 지중온도(°C)')
test_df = add_difference_column(test_df,   '5cm 지중온도(°C)', '30cm 지중온도(°C)')

In [120]:
train_df['시간_20to21_yn'] = train_df['시간'].isin([20,21]).astype(int)
test_df['시간_20to21_yn'] = test_df['시간'].isin([20,21]).astype(int)

In [121]:
drop_cols = ['date','시간', '역사명', '호선']
train_df = train_df.drop(columns=drop_cols, errors='ignore')
test_df = test_df.drop(columns=drop_cols, errors='ignore')

In [122]:
train_df.sample(5)

,일시,역명_호선,공기질,승차수,하차수,승하차수,기온(°C),강수량(mm),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),weekday,holi_yn,sat_yn,sun_yn,workday_yn,holiday_yn,workday_continue,holiday_continue,hour,month,quarter,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos,quarter_sin,quarter_cos,승차수_lag1,하차수_lag1,승하차수_lag1,승차수_lag2,하차수_lag2,승하차수_lag2,승차수_lag3,하차수_lag3,승하차수_lag3,승하차수_최근3시간평균,승하차수_최근3시간비율,승하차수_최근3시간증감률(%),기온(°C)_lag1,강수량(mm)_lag1,풍속(m/s)_lag1,습도(%)_lag1,증기압(hPa)_lag1,이슬점온도(°C)_lag1,현지기압(hPa)_lag1,해면기압(hPa)_lag1,일조(hr)_lag1,일사(MJ/m2)_lag1,적설(cm)_lag1,지면온도(°C)_lag1,5cm 지중온도(°C)_lag1,10cm 지중온도(°C)_lag1,20cm 지중온도(°C)_lag1,30cm 지중온도(°C)_lag1,기온(°C)_lag2,강수량(mm)_lag2,풍속(m/s)_lag2,습도(%)_lag2,증기압(hPa)_lag2,이슬점온도(°C)_lag2,현지기압(hPa)_lag2,해면기압(hPa)_lag2,일조(hr)_lag2,일사(MJ/m2)_lag2,적설(cm)_lag2,지면온도(°C)_lag2,5cm 지중온도(°C)_lag2,10cm 지중온도(°C)_lag2,20cm 지중온도(°C)_lag2,30cm 지중온도(°C)_lag2,기온(°C)_lag3,강수량(mm)_lag3,풍속(m/s)_lag3,습도(%)_lag3,증기압(hPa)_lag3,이슬점온도(°C)_lag3,현지기압(hPa)_lag3,해면기압(hPa)_lag3,일조(hr)_lag3,일사(MJ/m2)_lag3,적설(cm)_lag3,지면온도(°C)_lag3,5cm 지중온도(°C)_lag3,10cm 지중온도(°C)_lag3,20cm 지중온도(°C)_lag3,30cm 지중온도(°C)_lag3,기온(°C)_mean3h,기온(°C)_ratio3h,기온(°C)_diff3h,강수량(mm)_mean3h,강수량(mm)_ratio3h,강수량(mm)_diff3h,풍속(m/s)_mean3h,풍속(m/s)_ratio3h,풍속(m/s)_diff3h,습도(%)_mean3h,습도(%)_ratio3h,습도(%)_diff3h,증기압(hPa)_mean3h,증기압(hPa)_ratio3h,증기압(hPa)_diff3h,이슬점온도(°C)_mean3h,이슬점온도(°C)_ratio3h,이슬점온도(°C)_diff3h,현지기압(hPa)_mean3h,현지기압(hPa)_ratio3h,현지기압(hPa)_diff3h,해면기압(hPa)_mean3h,해면기압(hPa)_ratio3h,해면기압(hPa)_diff3h,일조(hr)_mean3h,일조(hr)_ratio3h,일조(hr)_diff3h,일사(MJ/m2)_mean3h,일사(MJ/m2)_ratio3h,일사(MJ/m2)_diff3h,적설(cm)_mean3h,적설(cm)_ratio3h,적설(cm)_diff3h,지면온도(°C)_mean3h,지면온도(°C)_ratio3h,지면온도(°C)_diff3h,5cm 지중온도(°C)_mean3h,5cm 지중온도(°C)_ratio3h,5cm 지중온도(°C)_diff3h,10cm 지중온도(°C)_mean3h,10cm 지중온도(°C)_ratio3h,10cm 지중온도(°C)_diff3h,20cm 지중온도(°C)_mean3h,20cm 지중온도(°C)_ratio3h,20cm 지중온도(°C)_diff3h,30cm 지중온도(°C)_mean3h,30cm 지중온도(°C)_ratio3h,30cm 지중온도(°C)_diff3h,주중주말,운행횟수,혼잡도,현지기압(hPa)_minus_해면기압(hPa),5cm 지중온도(°C)_minus_30cm 지중온도(°C),시간_20to21_yn
71089,2022-02-12 01:00:00,잠실_2,1.0,0.0,0.0,0.0,2.7,0.0,1.4,74.0,5.5,-1.4,1016.0,1026.8,0.0,0.00,0.0,-1.2,0.3,-0.3,-0.4,0.0,5,0,1,0,0,1,0,1,1,2,1,0.258819,0.965926,-0.974928,-0.222521,0.866025,5.000000e-01,1.000000e+00,6.123234e-17,0.0,0.0,0.0,335.0,683.0,1018.0,2167.0,1825.0,3992.0,1670.000000,0.0,-100.0,3.0,0.0,1.5,68.0,5.2,-2.3,1016.0,1026.8,0.0,0.00,0.0,-1.5,0.3,-0.3,-0.4,0.0,3.5,0.0,1.9,65.0,5.1,-2.4,1015.7,1026.4,0.0,0.00,0.0,-1.1,0.3,-0.3,-0.4,0.0,3.8,0.0,2.8,66.0,5.3,-1.9,1015.5,1026.2,0.0,0.00,0.0,-0.8,0.3,-0.3,-0.4,0.0,3.066667,0.880435,-1.1,0.0,NaN,0.0,1.600000,0.875000,-1.4,69.000000,1.072464,8.0,5.266667,1.044304,0.2,-2.033333,0.688525,0.5,1015.900000,1.000098,0.5,1026.666667,1.000130,0.6,0.000000,NaN,0.0,0.00,NaN,0.00,0.0,NaN,0.0,-1.266667,0.947368,-0.4,0.300000,1.000000,0.0,-0.300000,1.000000,0.0,-0.400000,1.000000,0.0,0.000000,NaN,0.0,SAT,0.0,0.000000,-10.8,0.3,0
82385,2022-05-28 17:00:00,홍대입구_2,1.0,5937.0,7080.0,13017.0,27.8,0.0,3.9,45.0,16.7,14.7,994.6,1004.2,1.0,1.89,0.0,39.1,29.1,26.2,23.2,21.7,5,0,1,0,0,1,0,1,17,5,2,-0.965926,-0.258819,-0.974928,-0.222521,0.500000,-8.660254e-01,1.224647e-16,-1.000000e+00,5731.0,7448.0,13179.0,4611.0,7267.0,11878.0,3559.0,7886.0,11445.0,12167.333333,1.069832,6.983179,28.4,0.0,5.1,39.0,15.1,13.1,994.5,1004.1,1.0,2.57,0.0,44.1,29.6,25.9,22.8,21.5,28.7,0.0,6.2,34.0,13.3,11.2,994.4,1003.9,1.0,3.10,0.0,49.3,29.4,25.4,22.3,21.3,28.7,0.0,5.1,40.0,15.7,13.7,994.3,1003.8,1.0,3.28,0.0,53.1,28.7,24.6,21.8,21.2,28.300000,0.982332,-0.9,0.0,NaN,0.0,5.066667,0.769737,-1.2,39.333333,1.144068,5.0,15.033333,1.110865,1.0,13.000000,1.130769,1.0,994.500000,1.000101,0.3,1004.066667,1.000133,0.4,1.000000,1.0,0.0,2.52,0.750000,-1.39,0.0,NaN,0.0,44.166667,0.885283,-14.0,29.366667,0.990919,0.4,25.833333,1.014194,1.6,22.766667,1.019034,1.4,21.500000,1.009302,0.5,SAT,22.0,591.681818,-9.6,7.4,0
27667,2022-02-27 19:00:00,사당_4,0.0,

In [ ]:
print(train_df.columns

Index(['일시', '역명_호선', '공기질', '승차수', '하차수', '승하차수', '기온(°C)', '강수량(mm)',
       '풍속(m/s)', '습도(%)',
       ...
       '20cm 지중온도(°C)_diff3h', '30cm 지중온도(°C)_mean3h', '30cm 지중온도(°C)_ratio3h',
       '30cm 지중온도(°C)_diff3h', '주중주말', '운행횟수', '혼잡도',
       '현지기압(hPa)_minus_해면기압(hPa)', '5cm 지중온도(°C)_minus_30cm 지중온도(°C)',
       '시간_20to21_yn'],
      dtype='object', length=155)


In [138]:
len(new_columns)

155

In [130]:
new_columns = [
    '일시', '역명_호선', '공기질', '승차수', '하차수', '승하차수',
    '기온_C', '강수량_mm', '풍속_m_s', '습도', '증기압_hPa', '이슬점온도_C', '현지기압_hPa', '해면기압_hPa',
    '일조_hr', '일사_MJ_m2', '적설_cm', '지면온도_C', '5cm_지중온도_C', '10cm_지중온도_C', '20cm_지중온도_C', '30cm_지중온도_C',
    'weekday', 'holi_yn', 'sat_yn', 'sun_yn', 'workday_yn', 'holiday_yn',
    'workday_continue', 'holiday_continue',
    'hour', 'month', 'quarter',
    'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
    '승차수_lag1', '하차수_lag1', '승하차수_lag1',
    '승차수_lag2', '하차수_lag2', '승하차수_lag2',
    '승차수_lag3', '하차수_lag3', '승하차수_lag3',
    '승하차수_최근3시간평균', '승하차수_최근3시간비율', '승하차최근3h증감',
    '기온_C_lag1', '강수량_mm_lag1', '풍속_m_s_lag1', '습도_lag1', '증기압_hPa_lag1', '이슬점온도_C_lag1', '현지기압_hPa_lag1', '해면기압_hPa_lag1',
    '일조_hr_lag1', '일사_MJ_m2_lag1', '적설_cm_lag1', '지면온도_C_lag1', '5cm_지중온도_C_lag1', '10cm_지중온도_C_lag1', '20cm_지중온도_C_lag1', '30cm_지중온도_C_lag1',
    '기온_C_lag2', '강수량_mm_lag2', '풍속_m_s_lag2', '습도_lag2', '증기압_hPa_lag2', '이슬점온도_C_lag2', '현지기압_hPa_lag2', '해면기압_hPa_lag2',
    '일조_hr_lag2', '일사_MJ_m2_lag2', '적설_cm_lag2', '지면온도_C_lag2', '5cm_지중온도_C_lag2', '10cm_지중온도_C_lag2', '20cm_지중온도_C_lag2', '30cm_지중온도_C_lag2',
    '기온_C_lag3', '강수량_mm_lag3', '풍속_m_s_lag3', '습도_lag3', '증기압_hPa_lag3', '이슬점온도_C_lag3', '현지기압_hPa_lag3', '해면기압_hPa_lag3',
    '일조_hr_lag3', '일사_MJ_m2_lag3', '적설_cm_lag3', '지면온도_C_lag3', '5cm_지중온도_C_lag3', '10cm_지중온도_C_lag3', '20cm_지중온도_C_lag3', '30cm_지중온도_C_lag3',
    '기온_C_mean3h', '기온_C_ratio3h', '기온_C_diff3h',
    '강수량_mm_mean3h', '강수량_mm_ratio3h', '강수량_mm_diff3h',
    '풍속_m_s_mean3h', '풍속_m_s_ratio3h', '풍속_m_s_diff3h',
    '습도_mean3h', '습도_ratio3h', '습도_diff3h',
    '증기압_hPa_mean3h', '증기압_hPa_ratio3h', '증기압_hPa_diff3h',
    '이슬점온도_C_mean3h', '이슬점온도_C_ratio3h', '이슬점온도_C_diff3h',
    '현지기압_hPa_mean3h', '현지기압_hPa_ratio3h', '현지기압_hPa_diff3h',
    '해면기압_hPa_mean3h', '해면기압_hPa_ratio3h', '해면기압_hPa_diff3h',
    '일조_hr_mean3h', '일조_hr_ratio3h', '일조_hr_diff3h',
    '일사_MJ_m2_mean3h', '일사_MJ_m2_ratio3h', '일사_MJ_m2_diff3h',
    '적설_cm_mean3h', '적설_cm_ratio3h', '적설_cm_diff3h',
    '지면온도_C_mean3h', '지면온도_C_ratio3h', '지면온도_C_diff3h',
    '5cm_지중온도_C_mean3h', '5cm_지중온도_C_ratio3h', '5cm_지중온도_C_diff3h',
    '10cm_지중온도_C_mean3h', '10cm_지중온도_C_ratio3h', '10cm_지중온도_C_diff3h',
    '20cm_지중온도_C_mean3h', '20cm_지중온도_C_ratio3h', '20cm_지중온도_C_diff3h',
    '30cm_지중온도_C_mean3h', '30cm_지중온도_C_ratio3h', '30cm_지중온도_C_diff3h',
    '주중주말', '운행횟수', '혼잡도',
    '현지기압_minus_해면기압', '5to30cm_지중온도',
    '시간_20to21_yn'
]


# 컬럼명 전체 변경
train_df.columns = new_columns
test_df.columns = new_columns


In [131]:
# train_df.to_excel('./data2/train.xlsx', index=False)
# test_df = test_df.dropna(subset='공기질')
# test_df.to_excel( './data2/test.xlsx', index=False)

In [132]:
train_df.to_csv('./data2/train_159cols.csv', encoding='utf-8', index=False)
test_df = test_df.dropna(subset='공기질')
test_df .to_csv( './data2/test_159cols.csv', encoding='utf-8', index=False)

In [137]:
test_1920Y_df = test_df[test_df['시간_20to21_yn'] == 1].copy()
test_1920N_df = test_df[test_df['시간_20to21_yn'] == 0].copy()
test_1920Y_df .to_csv( './data2/test_1920Y.csv', encoding='utf-8', index=False)
test_1920N_df .to_csv( './data2/test_1920N.csv', encoding='utf-8', index=False)


In [ ]:
# sns.heatmap(train_df.isna())